In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.
c:\users\skarkosz\appdata\local\continuum\anaconda3\envs\snakes\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\skarkosz\appdata\local\continuum\anaconda3\envs\snakes\lib\site-packages\tensorflow\python\framework\dtypes.py:472: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\skarkosz\appdata\local\continuum\anaconda3\envs\snakes\lib\site-packages\tensorflow\python\framework\dtypes.py:473: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 =

In [2]:

with open('dataset.txt','r',encoding='utf-8') as txt:
    text = txt.read()
print('corpus length:', len(text))

corpus length: 1762846


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 151


In [4]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 587602


In [5]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [8]:
y.shape

(587602, 151)

In [23]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))


Build model...


In [24]:
# # build the model: a single LSTM
# print('Build model...')
# model = Sequential()
# # model.add(LSTM(128, input_shape=(maxlen, len(chars))))
# model.add(Dense(len(chars), activation='softmax', input_shape=(maxlen, len(chars))))
# model.add(LSTM(len(chars)))
# model.add(Dense(len(chars), activation='softmax'))


In [25]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [26]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])

Epoch 1/20
587520/587602 [============================>.] - ETA: 0s - loss: 2.2401
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "iche powietrzne ogromy,
usiadł na skale,"
iche powietrzne ogromy,
usiadł na skale, w to w polegnie o to na wierzy

















































































































































































































































































































































































----- diversity: 0.5
----- Generating with seed: "iche powietrzne ogromy,
usiadł na skale,"
iche powietrzne ogromy,
usiadł na skale, nie naszym się pod wystawie,
                                                                                                                                                                                         

że żaden w parku tym nie było ścista,
I mojowa — To, niedochorzeg wielke cało więci,
wylata, wiądam murasz, jak łacka wsierce na anicji.
Nio fladen kradzie wzbudzącymi, dłońce,
The gleneńkz mojej błękitone, zagnieciego przy balgiegowem stukom o kultylkiem a natelcrzach.

Nie do strał nie maś gołębymy siłce w dawności,
Marzyć das zakrytąć, jak błyskawa traszu.
Oto pójdzie się nia wchleż w Mazy chłodno głębskie,
ziele nadobca
----- diversity: 1.2
----- Generating with seed: "zysty lazur,
że żaden w parku tym nie by"
zysty lazur,
że żaden w parku tym nie było, .

Tu jak pótvis:

tak roce, i gwiazd mówmańc gaćpa mą nie darhanka,
Wszystem świat kolelurkwomypośa…

Potesziaż olska móje,
czyższąca wróg, gdy lutęowsnienych I żebrą!
Więcej? /eców?, a
Lecz a wycark, którym z wytreszy, czeniony pruchuł!

alyteskacą — — prówa perary gńskim boł ęhem!»

Grośf pręci widzy, —
Pychcze śmierzonaciejąc nic niegłarć,
do i ich słążawce zlagoli
Tak! dup i Chuduchą. —
M
587602/587602 [========================

    Nie wielkie się to szczerze pod niewielkie do niebie,
    W pod niebie przez miejsce w szczerzy pod niebie,
                                                                                                                                                                                                                                                                                                    
----- diversity: 0.5
----- Generating with seed: "imo szarf,
Zawiązanych brylantami —
    "
imo szarf,
Zawiązanych brylantami —
    W cieni i się to widział pod kościecznie moja w nie czas szczebie.
































                                                                                                                                                                                                                                                                                                             
----- diversity: 1.0
----- Generating with seed: "imo szarf,
Z

c:\users\skarkosz\appdata\local\continuum\anaconda3\envs\snakes\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


bieg pił szybnie gdujesz poczucia ścię
notego się lujecie kruczju








----- diversity: 1.2
----- Generating with seed: "óre w nieznanej czekają krainie!

I wsze"
óre w nieznanej czekają krainie!

I wszednia blask okaz ie
jest przed z zbrysta duch jestą,
Na ciebie mupikł, roszu nu iwężejeskiem,
Cichoki twojego ont pierwych irau
suchycom rozcecnym zeglasto.
— Niemigna. Na
leżego, wznos groszej gorzę, na tych spud.





Zuesintecka

Bojacze w na nicurei ichacra,
Zaróntałem cały żylu.
Byłam niecopą w comą synkowania powiekać się
Z fan*we drzwi jak darłem
zadwali kostpozndarczem 

Mrancz fanącies,
Jó
587602/587602 [==============================] - 483s 823us/step - loss: 1.9474
Epoch 10/20
587520/587602 [============================>.] - ETA: 0s - loss: 1.9609
----- Generating text after Epoch: 9
----- diversity: 0.2
----- Generating with seed: "ina;
Ale z Filem psotnik Miluś
Bijatykę "
ina;
Ale z Filem psotnik Miluś
Bijatykę pod nie pod starem na pod nasze brzegi,
    Pod starem się w

w swej to strzało się w domu strzegi strzegi,
    Powietrze w nie do końce na domu w tej wielkiego w domu z nas nie zapatem strzeli.






















----- diversity: 0.5
----- Generating with seed: "owo narodzonemu



Jakże takimi dłońmi u"
owo narodzonemu



Jakże takimi dłońmi uwierzą.

Choć przestrzenia w swej wysłoku i powierzy Noe.

Zawieszona z zabrato przed stronowaną od pomną zastajne koły w czy nie ciemnej,
Słowa pokór skończy z szalonie od nadzieją słowie
I na słony wierzy starość,
                       twoje przed nóż,
mężę się jak potrzymał mi z nadzieją
o nie mi jedno w pieśnich na chmuru od dzikiej kusia,
z przegiaza jest schopił w przedzień w tukiem,







----- diversity: 1.0
----- Generating with seed: "owo narodzonemu



Jakże takimi dłońmi u"
owo narodzonemu



Jakże takimi dłońmi ustożem
sypił
dwoje niezływa i spustał pomorza
bałemieć dosłyszenicnie spoczy rozprzyłaÿűèÇ.à ücówek
muuśmija sun ciemności ci niech kłaczy…
Czy odpowie sława stwor, dobrzona.
Niedn

Niespostogidę w kobieczek widacz,aż grobu.
Dobrą skrążyć lud lrońca…


----- diversity: 1.2
----- Generating with seed: "rzyma
stąpał wyniośle z nonszalanckim ge"
rzyma
stąpał wyniośle z nonszalanckim genianki,
u szybiem w tą wszystko.żoniąceki, o — burze każdy utobéj céłaz…
    Byle Posiafą twojąé poéł lśniąc pożyje.
smąc. wkąd jak brąkry leje,
O Pająk się w kochyli,
dużoéj niszczęków, raz, a jyszeéj szczęśbik I jakbyś tak już wyżyu, we rekàsenj°é wie é?Ykd — szczamiem
Przymkniętywali syąskami brzałor

sługo  żógi przy męlnięzkounnymé kzetuj biała mé?
Nie ciebiej dół trwały, choć nigóstkać…

stó
587602/587602 [==============================] - 479s 816us/step - loss: 2.2518
Epoch 17/20
587520/587602 [============================>.] - ETA: 0s - loss: 2.8635
----- Generating text after Epoch: 16
----- diversity: 0.2
----- Generating with seed: "a,
Donosicielko chyża, matko folksdojcze"
a,
Donosicielko chyża, matko folksdojcze się w pod swoje się pod niebé w szé pod niebie
















Nié przed się pék się éńér się w éénie w podobnie w tak na się się przed na kocham éńéu,
Nie powieczne podobnie za się na podobnie na ném éń podoéńékń
----- diversity: 0.5
----- Generating with seed: "nieczystości chodzę
od domu do domu a za"
nieczystości chodzę
od domu do domu a za jeé nie ééniéj éńca — dutany taktie,
    Waliony przysłanie, na nie wieé porowoz.énie,
po karzmie — za dusze na dzieci stomnie tékéj



Przed wierza na éporowali, w miał w czekany siée już pierwszéj raz —
éé? Podéle? — Droga uczynie éźwiacznych pomieszami w nieéo podobna,
co ééé próżnieé na gdzie takiem,
éńca strumienie wé się dziéé;
Jak éputrze w duséééj stronący.

éźwiadło nie éń
światło na nie
----- diversity: 1.0
----- Generating with seed: "nieczystości chodzę
od domu do domu a za"
nieczystości chodzę
od domu do domu a zamieéu

  éénąékiégńska, i wierzeli zdomku wa ktakolibekoi.
Podolnyé w nagrwaéjéj konóéb.ékép, choć hyét?



 Sé;éże wrzzetowanący, każkiszczet, co do mnie, éźwięteéén stamé,
éńca lodu 

In [27]:

neural_texts = list()
fan_fic_len = 400

for n in range(10):
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.7]:

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence

        for i in range(fan_fic_len):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    neural_texts.append(generated)

c:\users\skarkosz\appdata\local\continuum\anaconda3\envs\snakes\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


In [16]:
neural_texts

['\no! przyświećcie blaskiem skrzydeł swoich wziele;\nTa przybo moc i sków słuchach\nTu świerowa słunech dziej\nCieha się wszystkie liść\njak mi jud wieków płynie przyadli zgiśliech spiarą jak dziernące wysólida,\nA śmierć nigu nas wygo bosem słoszeczie.\n\nNiece z ucholna.\n\nNie było chłoskie ust wielką ze spała,\nNiebeju na wieć poniosłym wiedną się bając,\nprzyświ tak w mu sarzał się czarną okragitem,\nnie brzmierzały w uchrów,\nNo jeze samnonych j',
 ' przyczyny,\nSwych bliźnich zawsze sądzim potrzebie.\n\nNiech kurtonia przywatem?\n\nNiech żypają ziemiał śmiersą w krygnienie wraje,\nscerziwe znawiste, i trzech ci się wskija znajdzie,\nnie będzie obrozne, zdybrago,\nZemsnąc poczy wydobne drogi.\n\ntyn głaz wytazi się w chcerust lude brzy\nto oto, ledzą zmieniali i tak w sobie\n\n\n\nI\n\n\n\nZiebie imy strzele —\nniu nie jedność który w nim zielią.\n\nNie drzenie spracował ludzki poznonia\n\n\n\nO nowy, co ro',
 'ę w wnętrznościach twych duch struchlałych —\nNiech potraba,\nBy w ś

In [32]:
model.save('.\model1.h5')
model.save_weights('.\model1Weights.h5')